In [1]:
## 1. Import Thư Viện Cần Thiết
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# KHÔNG DÙNG train_test_split NỮA
import os
import sys

# Tăng giới hạn đệ quy để tránh lỗi khi lưu file pickle lớn
sys.setrecursionlimit(3000) 

# Danh sách Stop Words Tiếng Việt cơ bản (Dùng để tối ưu TfidfVectorizer)
VIETNAMESE_STOP_WORDS = [
    'là', 'và', 'của', 'với', 'những', 'các', 'một', 'được', 'cho', 'này', 
    'kia', 'rằng', 'thì', 'như', 'vì', 'ở', 'từ', 'bên', 'ra', 'vào', 'bị'
]

In [2]:
## 2. Tải Dữ Liệu Đã Xử Lý
try:
    df = pd.read_csv('data_EDA.csv')
    print("Đã tải file data_EDA.csv thành công.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file data_EDA.csv. Vui lòng chạy notebook 2 trước.")
    exit()

print(f"\nKích thước dữ liệu ban đầu: {df.shape}") # Hiện tại là (2000, X)


Đã tải file data_EDA.csv thành công.

Kích thước dữ liệu ban đầu: (50000, 7)


In [3]:
## 3. GIẢM SỐ LƯỢNG BÀI HÁT XUỐNG MỨC AN TOÀN TUYỆT ĐỐI (2000 bài)

# LẤY CHÍNH XÁC 2000 DÒNG ĐẦU TIÊN để xây dựng mô hình
df_model = df.head(2000).copy() 
df_model = df_model.reset_index(drop=True)
print(f"Kích thước tập Mô hình (Đã giảm): {df_model.shape}") 
# Kích thước này BẮT BUỘC phải là (2000, X)


Kích thước tập Mô hình (Đã giảm): (2000, 7)


In [4]:
## 4. Xây Dựng Mô Hình (TF-IDF Vectorizer và Cosine Similarity Matrix)

# **Bước 1: Khởi tạo và Huấn luyện TfidfVectorizer**
# Tối ưu: max_features=1000 và min_df=2
tfidf = TfidfVectorizer(
    stop_words=VIETNAMESE_STOP_WORDS,
    max_features=1000,                
    max_df=0.8,                      
    min_df=2                          
)

print("\nBắt đầu huấn luyện TfidfVectorizer trên 2000 bài hát...")
# Fit and transform trên trường 'Content'
tfidf_matrix = tfidf.fit_transform(df_model['Content'])
print(f"Kích thước Ma trận TF-IDF: {tfidf_matrix.shape}") 
print(f"Số lượng từ vựng (features) thực tế được sử dụng: {len(tfidf.vocabulary_)}")


Bắt đầu huấn luyện TfidfVectorizer trên 2000 bài hát...
Kích thước Ma trận TF-IDF: (2000, 141)
Số lượng từ vựng (features) thực tế được sử dụng: 141


In [5]:
# **Bước 2: Tính toán Ma trận Tương Đồng Cosine và tối ưu kiểu dữ liệu**
print("Bắt đầu tính toán Ma trận Tương Đồng Cosine...")

# Kích thước ma trận 2000 x 2000. Dùng float32 (khoảng 16 MB)
cosine_sim_float64 = cosine_similarity(tfidf_matrix)
cosine_sim = cosine_sim_float64.astype(np.float32) 

print(f"Kích thước Ma trận Tương Đồng Cosine: {cosine_sim.shape}") 
print(f"Kiểu dữ liệu Ma trận: {cosine_sim.dtype}") 

Bắt đầu tính toán Ma trận Tương Đồng Cosine...
Kích thước Ma trận Tương Đồng Cosine: (2000, 2000)
Kiểu dữ liệu Ma trận: float32


In [6]:
## 5. Chuẩn Bị Dữ Liệu Phụ Trợ (Ánh xạ Title và Index)
indices_model = pd.Series(df_model.index, index=df_model['Title'])

## 6. Lưu Mô Hình và Các Thành Phần Cần Thiết (.pkl)
model_components = {
    'tfidf_vectorizer': tfidf,
    'cosine_sim_matrix': cosine_sim,
    'song_indices': indices_model, 
    'df_model': df_model[['SongID', 'Title', 'Artist', 'Genre']] 
}

try:
    with open('model.pkl', 'wb') as f:
        pickle.dump(model_components, f)
    print("\nĐã lưu mô hình thành công vào file: model.pkl")
    
except Exception as e:
    print(f"Lỗi khi lưu file: {e}")


Đã lưu mô hình thành công vào file: model.pkl


In [7]:
## 5. Chuẩn Bị Dữ Liệu Phụ Trợ (Ánh xạ Title và Index)
# Tạo Series ánh xạ Title tới vị trí Index
indices_model = pd.Series(df_model.index, index=df_model['Title'])